In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import seaborn as sns; sns.set(style="ticks", color_codes="pastel")
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
def stdscaler(df):
    scaler = StandardScaler()
    scaler.fit(df)
    scX_df = scaler.transform(df)
    df=pd.DataFrame(scX_df, index =df.index,columns =df.columns  )
    return df
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
file_name = 'UCI_Credit_Card 2.csv'
data =pd.read_csv(file_name,index_col = 0)
data

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
2,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
3,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
4,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
5,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000.0,1,3,1,39,0,0,0,0,0,...,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29997,150000.0,1,3,2,43,-1,-1,-1,-1,0,...,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29998,30000.0,1,2,2,37,4,3,2,-1,0,...,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1


In [2]:
print(data.LIMIT_BAL.describe())
conditions = [(data.LIMIT_BAL<=50000),
              (data.LIMIT_BAL>50000)&(data.LIMIT_BAL<=140000),
             (data.LIMIT_BAL>140000)&(data.LIMIT_BAL<=240000),
             (data.LIMIT_BAL>240000),]
choices = ['limlow', 'limmedium', 'limhigh','limexhigh']
data['limclass'] = np.select(conditions, choices, default='limlow')
data['limclass']

count      30000.000000
mean      167484.322667
std       129747.661567
min        10000.000000
25%        50000.000000
50%       140000.000000
75%       240000.000000
max      1000000.000000
Name: LIMIT_BAL, dtype: float64


ID
1           limlow
2        limmedium
3        limmedium
4           limlow
5           limlow
           ...    
29996      limhigh
29997      limhigh
29998       limlow
29999    limmedium
30000       limlow
Name: limclass, Length: 30000, dtype: object

In [3]:
print(data.AGE.describe())
conditions = [(data.AGE<=30),
              (data.AGE>30)&(data.AGE<=40),
             (data.AGE>40)&(data.AGE<=50),
              (data.AGE>50)&(data.AGE<=60),
              (data.AGE>60)&(data.AGE<=70),
             (data.AGE>70),]
choices = ['20s', '30s', '40s','50s','60s','70s']
data['ageclass'] = np.select(conditions, choices, default='20s')
data['ageclass']

count    30000.000000
mean        35.485500
std          9.217904
min         21.000000
25%         28.000000
50%         34.000000
75%         41.000000
max         79.000000
Name: AGE, dtype: float64


ID
1        20s
2        20s
3        30s
4        30s
5        50s
        ... 
29996    30s
29997    40s
29998    30s
29999    40s
30000    40s
Name: ageclass, Length: 30000, dtype: object

In [4]:
pay_nm =['PAY_0', 'PAY_2','PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
dubo = data.copy()

dubo[pay_nm]=np.where(dubo[pay_nm]>=1,1,0)
    
data["paystate"] = dubo[pay_nm].sum(axis = 1)
data['lastpay'] =dubo['PAY_0']

In [5]:
data['totbill']=data[['BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']].sum(axis=1)
data['totalpay']=data[['PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']].sum(axis=1)         

data=data.loc[data['totbill']!=0].copy()
data['billpay']=(data['totbill']-data['totalpay'])/data['totbill']
data['billlim']=data['totbill']/6/data['LIMIT_BAL']

In [6]:
data['billlim']=data['totbill']/6/data['LIMIT_BAL']
data['paylim']=data['totalpay']/6/data['LIMIT_BAL']

In [7]:
pay_nm =['PAY_0', 'PAY_2','PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
dubo = data.copy()

dubo[pay_nm]=np.where(dubo[pay_nm]==0,1,0)

data['paymin'] =dubo[pay_nm].sum(axis = 1)

dubo = data.copy()

dubo[pay_nm]=np.where(dubo[pay_nm]==-1,1,0)

data['payful'] =dubo[pay_nm].sum(axis = 1)

dubo = data.copy()
dubo[pay_nm]=np.where(dubo[pay_nm]==-2,1,0)

data['inactive'] =dubo[pay_nm].sum(axis = 1)

In [8]:
bill_ix =['BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
repay_ix =['PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
for i in range(0,5):
    data['billchangelim'+str(i)]=(data[bill_ix[i+1]]-data[bill_ix[i]])/data['LIMIT_BAL']
    data['paychangelim'+str(i)]=(data[repay_ix[i+1]]-data[repay_ix[i]])/data['LIMIT_BAL']


In [9]:
agedummy=pd.get_dummies(data.ageclass,prefix='ageclass')
limdummy=pd.get_dummies(data.limclass,prefix='limclass')
sexdummy =pd.get_dummies(data.SEX,prefix='sex')
edudummy=pd.get_dummies(data.EDUCATION,prefix='education')
mardummy =pd.get_dummies(data.MARRIAGE,prefix='marriage')

#data = pd.concat([data,sexdummy,edudummy,mardummy ], axis=1, sort=False)
print(data.columns)
data_offline =data[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',"default.payment.next.month"]].copy()
bill_ix =['BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
repay_ix =['PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
com_ix =['Billpay1','Billpay2','Billpay3','Billpay4','Billpay5','Billpay6',]
#for c,b,r in zip(com_ix,bill_ix ,repay_ix):
#    data_offline[c]=data[b]-data[r]


Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month', 'limclass', 'ageclass', 'paystate',
       'lastpay', 'totbill', 'totalpay', 'billpay', 'billlim', 'paylim',
       'paymin', 'payful', 'inactive', 'billchangelim0', 'paychangelim0',
       'billchangelim1', 'paychangelim1', 'billchangelim2', 'paychangelim2',
       'billchangelim3', 'paychangelim3', 'billchangelim4', 'paychangelim4'],
      dtype='object')


In [10]:
"""print(data[['BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',]])"""

data

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,billchangelim0,paychangelim0,billchangelim1,paychangelim1,billchangelim2,paychangelim2,billchangelim3,paychangelim3,billchangelim4,paychangelim4
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,2,2,1,24,2,2,-1,-1,-2,...,-0.040550,0.034450,-0.120650,-0.034450,-0.034450,0.000000,0.000000,0.000000,0.000000,0.000000
2,120000.0,2,2,2,26,-1,2,0,0,0,...,-0.007975,0.008333,0.007975,0.000000,0.004917,0.000000,0.001525,-0.008333,-0.001617,0.016667
3,90000.0,2,2,2,34,0,0,0,0,0,...,-0.169022,-0.000200,-0.005200,-0.005556,0.008578,0.000000,0.006856,0.000000,0.006678,0.044444
4,50000.0,2,2,1,37,0,0,0,0,0,...,0.024860,0.000380,0.021160,-0.016380,-0.419540,-0.002000,0.012900,-0.000620,0.011760,-0.001380
5,50000.0,1,2,1,57,-1,0,-1,0,0,...,-0.058940,0.693620,0.603300,-0.533620,-0.297900,-0.020000,-0.035880,-0.166220,-0.000300,-0.000200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000.0,1,3,1,39,0,0,0,0,0,...,0.017577,0.052273,0.070682,-0.068168,-0.547095,-0.008891,-0.258032,0.008877,-0.069350,-0.018182
29997,150000.0,1,3,2,43,-1,-1,-1,-1,0,...,0.000967,0.011260,0.011160,0.036480,0.036513,-0.059127,-0.025260,-0.000860,-0.034600,0.000000
29998,30000.0,1,2,2,37,4,3,2,-1,0,...,-0.006967,0.000000,-0.019933,0.733333,0.604000,-0.593333,-0.009867,-0.073333,-0.040833,0.036667


In [11]:
model_1 =data[['paystate','lastpay','billpay', 'billlim', 'paylim',
               'billchangelim0', 'paychangelim0', 'billchangelim1', 
               'paychangelim1',
       'billchangelim2', 'paychangelim2', 
               'billchangelim3', 'paychangelim3',
       'billchangelim4', 'paychangelim4','paymin','payful',"inactive"]].copy()



model_1 = pd.concat([model_1,agedummy,sexdummy,edudummy,mardummy,data[["default.payment.next.month"]] ], axis=1, sort=False)
model_1.columns

Index(['paystate', 'lastpay', 'billpay', 'billlim', 'paylim', 'billchangelim0',
       'paychangelim0', 'billchangelim1', 'paychangelim1', 'billchangelim2',
       'paychangelim2', 'billchangelim3', 'paychangelim3', 'billchangelim4',
       'paychangelim4', 'paymin', 'payful', 'inactive', 'ageclass_20s',
       'ageclass_30s', 'ageclass_40s', 'ageclass_50s', 'ageclass_60s',
       'ageclass_70s', 'sex_1', 'sex_2', 'education_0', 'education_1',
       'education_2', 'education_3', 'education_4', 'education_5',
       'education_6', 'marriage_0', 'marriage_1', 'marriage_2', 'marriage_3',
       'default.payment.next.month'],
      dtype='object')

In [12]:
model_1.to_csv('model1.csv')

In [16]:
model_1

,paystate,lastpay,billpay,billlim,paylim,billchangelim0,paychangelim0,billchangelim1,paychangelim1,billchangelim2,...,education_2,education_3,education_4,education_5,education_6,marriage_0,marriage_1,marriage_2,marriage_3,default.payment.next.month
ID,,,,,,,,,,,,,,,,,,,,,
1,2,1,0.910566,0.064200,0.005742,-0.040550,0.034450,-0.120650,-0.034450,-0.034450,...,1,0,0,0,0,0,1,0,0,1
2,2,0,0.707209,0.023718,0.006944,-0.007975,0.008333,0.007975,0.000000,0.004917,...,1,0,0,0,0,0,0,1,0,1
3,0,0,0.891612,0.188246,0.020404,-0.169022,-0.000200,-0.005200,-0.005556,0.008578,...,1,0,0,0,0,0,0,1,0,0
4,0,0,0.963741,0.771113,0.027960,0.024860,0.000380,0.021160,-0.016380,-0.419540,...,1,0,0,0,0,0,1,0,0,0
5,0,0,0.459946,0.364463,0.196830,-0.058940,0.693620,0.603300,-0.533620,-0.297900,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,0,0,0.941339,0.549507,0.032235,0.017577,0.052273,0.070682,-0.068168,-0.547095,...,0,1,0,0,0,0,1,0,0,0
29997,0,0,0.315929,0.023536,0.016100,0.000967,0.011260,0.011160,0.036480,0.036513,...,0,1,0,0,0,0,0,1,0,0
29998,3,1,0.556003,0.391644,0.173889,-0.006967,0.000000,-0.019933,0.733333,0.604000,...,1,0,0,0,0,0,0,1,0,1


In [14]:
model2 =data[['LIMIT_BAL','BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6','paymin', 'payful', 'inactive', 'billchangelim0', 'paychangelim0',
       'billchangelim1', 'paychangelim1', 'billchangelim2', 'paychangelim2',
       'billchangelim3', 'paychangelim3', 'billchangelim4', 'paychangelim4','default.payment.next.month']].copy()

model2.to_csv('model2.csv')

In [15]:
model2

,LIMIT_BAL,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,...,paychangelim0,billchangelim1,paychangelim1,billchangelim2,paychangelim2,billchangelim3,paychangelim3,billchangelim4,paychangelim4,default.payment.next.month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,...,0.034450,-0.120650,-0.034450,-0.034450,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,120000.0,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,...,0.008333,0.007975,0.000000,0.004917,0.000000,0.001525,-0.008333,-0.001617,0.016667,1
3,90000.0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,...,-0.000200,-0.005200,-0.005556,0.008578,0.000000,0.006856,0.000000,0.006678,0.044444,0
4,50000.0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,...,0.000380,0.021160,-0.016380,-0.419540,-0.002000,0.012900,-0.000620,0.011760,-0.001380,0
5,50000.0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,...,0.693620,0.603300,-0.533620,-0.297900,-0.020000,-0.035880,-0.166220,-0.000300,-0.000200,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000.0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,...,0.052273,0.070682,-0.068168,-0.547095,-0.008891,-0.258032,0.008877,-0.069350,-0.018182,0
29997,150000.0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,...,0.011260,0.011160,0.036480,0.036513,-0.059127,-0.025260,-0.000860,-0.034600,0.000000,0
29998,30000.0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,...,0.000000,-0.019933,0.733333,0.604000,-0.593333,-0.009867,-0.073333,-0.040833,0.036667,1
